# Train a neural network (velocity for ramps)

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Set seeds

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

Set parameters

In [ ]:
# file with training input and output data: format is x,y
input_file = 'RampData_MAR2_speed.csv'

# names of input/output columns
inputs = ['mu', 'length', 'angle']
outputs = ['speed']

# these set the input/output dimensions of the network
input_size = len(inputs)
output_size = len(outputs)

Read data

In [ ]:
data = pd.read_csv(input_file)
data.columns = inputs + outputs

x = np.array(data[inputs])
y = np.array(data[outputs])

Normalizing preprocessing layer from training data

In [ ]:
normalizer = keras.layers.experimental.preprocessing.Normalization()
normalizer.adapt(x)

Base model

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(input_size, activation='linear'))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(output_size,activation='relu'))
model.compile(loss='mse',optimizer='adam')

Compile model

In [ ]:
input_shape = x.shape[1:]
full_model_input = keras.Input(shape=input_shape)
normalized_input = normalizer(full_model_input)
full_model_output = model(normalized_input)
full_model = keras.Model(full_model_input, full_model_output)
full_model.compile(loss='mse',optimizer='adam',metrics=['mse'])

Split data

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.33, shuffle= True)

Train model

In [ ]:
model_output = full_model.fit(x_train,y_train,epochs=1000,batch_size=3,verbose=0,validation_data=(x_val,y_val)) # check validation

View training

In [ ]:
plt.yscale('log')
plt.title('loss')
plt.plot(model_output.history['loss'], label='train')
plt.plot(model_output.history['val_loss'], label='validation')
plt.legend()
plt.figure()
plt.yscale('log')
plt.title('mse')
plt.plot(model_output.history['mse'], label='train')
plt.plot(model_output.history['val_mse'], label='validation')
plt.legend()
plt.figure()

View output

In [ ]:
# plot speed vs angle given mu, threshold
mu = 1 # set mu
length = 1 # set length
angle = np.linspace(-0.6,0.65,30)

# bug: mu = 0.009 is read as 0.0090..01
#plot_values = [i for i in x if i[0] == mu and i[2] == thresh] # x, y
plot_x = [i for i in x if (np.isclose(i[0], mu) and np.isclose(i[1], length))] # x, y
# this is not generic enough...
pred_x = [[mu, length, a] for a in angle]
pred = full_model.predict(pred_x)

plt.scatter([i[2] for i in plot_x], [y[i] for i,v in enumerate(x) if (np.isclose(v[0], mu) and np.isclose(v[1], length))])
plt.scatter([i[2] for i in pred_x], pred)

Save model

In [ ]:
full_model.save('1_speed_ramp_network.h5')

Find largest divergence between prediction and training data

In [ ]:
# divergence = (full_model.predict(x) - y)
# max_divergence = max([i[1] for i in divergence])
# print(max_divergence)
# print(max_divergence, x[np.where(divergence == max_divergence)[0]])

Time predictions

In [ ]:
# %%timeit time_x = np.array([np.random.rand(3)])
# full_model.predict(time_x)

Make a lot of plots

In [ ]:
mus = data['mu'].unique()
lengths = data['length'].unique()
angles = data['angle'].unique()
a=0
# plot speed vs angle given mu, threshold
for mu in mus:
    for length in lengths:
        # bug: mu = 0.009 is read as 0.0090..01
        #plot_values = [i for i in x if i[0] == mu and i[2] == thresh] # x, y
        plot_x = [i for i in x if (np.isclose(i[0], mu) and np.isclose(i[1], length))] # x, y
        pred_x = [[mu,length,angle] for angle in np.linspace(angles.min(),angles.max(),30)]
        pred = full_model.predict(pred_x)
        #plt.plot([i[1] for i in plot_values], [y[i] for i,v in enumerate(x) if v[0] == mu and v[2] == thresh])
        fig = plt.figure()
        plt.title('mu: %.3f, length: %.2f' % (mu, length))
        plt.scatter([i[2] for i in plot_x], [y[i] for i,v in enumerate(x) if (np.isclose(v[0], mu) and np.isclose(v[1], length))],
                label = 'training')
        plt.plot([i[2] for i in pred_x], pred, label = 'predicted', color='orange', marker='o')
        plt.legend()
        plt.savefig('plots/mu-%.3f_length-%.2f.png' % (mu,length))
        plt.close()